# Simple Bollinger bands limit orders strategy with manual interaction

In [1]:
import numpy as np
from simple.plotly import interactFigure
from simple.funcs import vwap
import pandas as pd
from simple.backtest import getLong, getShort, npBacktestLimit, getProfit

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
len(T)

907307

In [3]:
def model(Period: int = (100, 50000, 100), StdDev: float = (1, 4, 0.1)):
    if Period == 0 or StdDev == 0: return 0, {}
    Tick = T.Price
    Center = vwap(T, Period)
    std = pd.Series(Tick).rolling(Period).std().bfill().values
    qA = Center + std*StdDev
    qB = Center - std*StdDev

    trades = npBacktestLimit(T, qA, qB)
    Buy, Sell = getLong(trades), getShort(trades)
    P = getProfit(trades)
    Profit = {'x': P.Index, 'y': P.Profit.cumsum()}

    return P.Profit.sum(), locals()

In [4]:
interactFigure(model,
    Tick={'color': 'gray', 'opacity': 0.1},
    Center={'color': 'blue', 'opacity': 0.4},
    qA={'color': 'red', 'opacity': 0.5, 'dash': 'dot'},
    qB={'color': 'green', 'opacity': 0.5, 'dash': 'dot'},
    Profit={'mode': 'lines', 'color': 'gray', 'width': 3, 'opacity': 0.5, 'secondary_y': True, 'shape': 'hv', 'connectgaps': True},
    Buy={'mode': 'markers', 'color': 'green', 'symbol': 'triangle-up', 'size': 10, 'line': {'color': 'darkgreen', 'width': 1}},
    Sell={'mode': 'markers', 'color': 'red', 'symbol': 'triangle-down', 'size': 10, 'line': {'color': 'darkred', 'width': 1}}
)